In [421]:
# pip install transformers

In [422]:
# pip install torch

In [423]:
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import torch
from torch.optim import AdamW
import numpy as np

In [424]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [425]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [426]:
model.to(device)
model.train()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), ep

In [427]:
# pip install datasets


In [428]:
from datasets import load_dataset

# Load the SQuAD dataset
dataset = load_dataset("squad", "plain_text", split="train")


Found cached dataset squad (/Users/shashwatbindal/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


In [429]:
train_features = []

for example in dataset:
    start_positions = []
    end_positions = []

    context = example["context"]
    questions = example["question"]
    answers = example["answers"]
    answer_start=0
    answer_end=0
    tokenized_context = tokenizer.tokenize(context)
    tokenized_question = tokenizer.tokenize(questions)
    tokenized_input = ["[CLS]"] + tokenized_question + ["[SEP]"] + tokenized_context + ["[SEP]"]
    inputs = tokenizer.convert_tokens_to_ids(tokenized_input)
    char_to_token = []
    token_index = 1  # Skip the [CLS] token
    for char_index, char in enumerate(context):
            if char != " " and char != "\n":
                char_to_token.append(token_index)
            if token_index < len(tokenized_input) - 1 and char_index + 1 < len(tokenized_input[token_index]):
                token_index += 1

    for answer in answers["answer_start"]:
        answer_start = answer
        token_start = char_to_token[min(answer_start, len(char_to_token) - 1)]
        token_start = min(token_start, len(inputs) - 1)
        start_positions.append(token_start)



    for text in answers["text"]:
        answer_end = answer_start + len(text) - 1
        token_end = char_to_token[min(answer_end, len(char_to_token) - 1)]
        token_end = min(token_end, len(inputs) - 1)
        end_positions.append(token_end)

    


In [430]:
train_features.append(
            {
                "input_ids": inputs,
                "attention_mask": [1] * len(inputs),
                "start_positions": start_positions,
                "end_positions": end_positions,
            }
        )

In [431]:
train_features

[{'input_ids': [101,
   2054,
   2003,
   2463,
   2278,
   2019,
   3988,
   2964,
   1997,
   1029,
   102,
   28045,
   4956,
   2103,
   1006,
   2463,
   2278,
   1007,
   1010,
   1999,
   2344,
   2000,
   5326,
   2248,
   4262,
   2038,
   2511,
   2019,
   2248,
   4262,
   15128,
   1006,
   20868,
   2278,
   1007,
   1012,
   2463,
   2278,
   1005,
   1055,
   2034,
   2248,
   3276,
   2001,
   2511,
   1999,
   3339,
   2007,
   1996,
   2103,
   1997,
   8207,
   1010,
   5392,
   1010,
   2142,
   2163,
   1012,
   2023,
   4023,
   2038,
   2042,
   2582,
   9412,
   2011,
   7411,
   5337,
   6550,
   2007,
   1022,
   2060,
   3655,
   1024,
   9587,
   10422,
   15319,
   2103,
   1997,
   2900,
   1010,
   10541,
   1997,
   1996,
   3915,
   1010,
   8675,
   2239,
   1006,
   3839,
   8369,
   7828,
   1007,
   1997,
   12620,
   1010,
   8418,
   1005,
   2019,
   1997,
   1996,
   2111,
   1005,
   1055,
   3072,
   1997,
   2859,
   1010,
   20790,
   1997,


In [432]:
# pip install wrapt


In [433]:
import torch
from transformers import BertForQuestionAnswering, AdamW
from torch.utils.data import DataLoader, RandomSampler
from transformers import AdamW
import torch.optim as optim

# Define the batch size and number of training epochs
batch_size = 4
num_epochs = 50

# Convert the train_features list to a PyTorch DataLoader
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor([f["input_ids"] for f in train_features], dtype=torch.long),
    torch.tensor([f["attention_mask"] for f in train_features], dtype=torch.long),
    torch.tensor([f["start_positions"] for f in train_features], dtype=torch.long),
    torch.tensor([f["end_positions"] for f in train_features], dtype=torch.long),
)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

# Load the pre-trained BERT model for question answering
model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

# Set the model to training mode
model.train()

# Define the optimizer and learning rate
# optimizer = AdamW(model.parameters(), lr=3e-6)
optimizer = optim.Adamax(model.parameters(), lr=3e-6)


In [434]:
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for batch in train_dataloader:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, start_positions, end_positions = batch
        
        optimizer.zero_grad()
        
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            start_positions=start_positions,
            end_positions=end_positions
        )
        
        loss = outputs.loss
        loss.backward()
        optimizer.step()

Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/50
Epoch 36/50
Epoch 37/50
Epoch 38/50
Epoch 39/50
Epoch 40/50
Epoch 41/50
Epoch 42/50
Epoch 43/50
Epoch 44/50
Epoch 45/50
Epoch 46/50
Epoch 47/50
Epoch 48/50
Epoch 49/50
Epoch 50/50


In [435]:
save_path = "/Users/shashwatbindal/Fine_Tuned_model"

# Save the fine-tuned model
model.save_pretrained(save_path)

In [436]:
from transformers import BertForQuestionAnswering, BertTokenizer

# Load the fine-tuned BERT model
model = BertForQuestionAnswering.from_pretrained(save_path)
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

# Prepare the input
question = "what is the importance of heart surgery?"
context = """Importance of Heart Surgery:
Heart surgery plays a crucial role in the management of heart diseases, including coronary artery disease, valvular disorders, congenital heart defects, and heart failure. It aims to restore normal heart function, improve blood flow, and alleviate symptoms, thereby enhancing the overall well-being of patients. Without surgical intervention, many heart conditions would have severe consequences and reduce life expectancy.

Common Heart Surgery Procedures:
a) Coronary Artery Bypass Grafting (CABG):
CABG is a commonly performed procedure to bypass blocked or narrowed coronary arteries. Surgeons utilize grafts, typically taken from the patient's own veins or arteries, to create alternative pathways for blood flow, restoring proper circulation to the heart muscle.

b) Valve Replacement/Repair:
Valvular heart diseases, such as aortic stenosis or mitral regurgitation, often require valve repair or replacement. Surgeons either repair the damaged valve or replace it with a mechanical or biological valve prosthesis, improving valve function and preventing further complications.

c) Congenital Heart Defect Repair:
Heart surgery also addresses congenital heart defects, which are present at birth. Surgeons perform procedures to correct structural abnormalities, allowing normal blood flow and reducing long-term complications. Examples include atrial septal defect (ASD) closure, ventricular septal defect (VSD) repair, and tetralogy of Fallot correction.

d) Heart Transplantation:
In cases of end-stage heart failure, heart transplantation becomes the last resort. Surgeons replace the diseased heart with a healthy donor heart, providing a new lease of life to individuals with severe cardiac conditions.

Advancements in Heart Surgery:
a) Minimally Invasive Techniques:
Technological advancements have facilitated the development of minimally invasive heart surgery. These procedures involve smaller incisions, resulting in reduced pain, shorter hospital stays, and faster recovery for patients. Minimally invasive approaches include robotic-assisted surgery and transcatheter interventions, such as transcatheter aortic valve replacement (TAVR)."""
inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)

start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Process the logits to obtain the predicted answer span
start_index = torch.argmax(start_logits)
end_index = torch.argmax(end_logits)

# Decode the predicted answer span
all_tokens = tokenizer.convert_ids_to_tokens(inputs.input_ids[0])
answer = tokenizer.convert_tokens_to_string(all_tokens[start_index:end_index+1])

# Print the predicted answer
print("Question:", question)
print("Predicted Answer:", answer)


Question: what is the importance of heart surgery?
Predicted Answer: heart surgery plays a crucial role in the management of heart diseases


In [437]:
from transformers import BertForQuestionAnswering, BertTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader
import torch

# Load the fine-tuned BERT model
model = BertForQuestionAnswering.from_pretrained(save_path)
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

# Load the SQuAD validation dataset
datasetv = load_dataset("squad", "plain_text", split="validation[:500]")


Found cached dataset squad (/Users/shashwatbindal/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


In [438]:
from transformers import BertForQuestionAnswering, BertTokenizer

# Load the fine-tuned BERT model
model = BertForQuestionAnswering.from_pretrained(save_path)
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

# # Prepare the input

# inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")

# # Perform inference
# with torch.no_grad():
#     outputs = model(**inputs)

# start_logits = outputs.start_logits
# end_logits = outputs.end_logits

# # Process the logits to obtain the predicted answer span
# start_index = torch.argmax(start_logits)
# end_index = torch.argmax(end_logits)

# # Decode the predicted answer span
# all_tokens = tokenizer.convert_ids_to_tokens(inputs.input_ids[0])
# answer = tokenizer.convert_tokens_to_string(all_tokens[start_index:end_index+1])
def generate_predictions(dataset, tokenizer, model):
    predictions = []
    for example in dataset:
        inputs = tokenizer.encode_plus(example["question"], example["context"], add_special_tokens=True, return_tensors="pt")
        input_ids = inputs["input_ids"].to(model.device)
        attention_mask = inputs["attention_mask"].to(model.device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            start_logits = outputs.start_logits
            end_logits = outputs.end_logits

        start_index = torch.argmax(start_logits)
        end_index = torch.argmax(end_logits)

        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][start_index:end_index + 1]))
        predictions.append(answer)

    return predictions


In [439]:
predictions = generate_predictions(datasetv, tokenizer, model)


In [440]:
predictions

['denver broncos',
 'carolina panthers',
 'san francisco bay area at santa clara , california',
 'denver broncos',
 'gold',
 'golden anniversary',
 'february 7 , 2016',
 'american football conference',
 'golden anniversary',
 'american football conference',
 'february 7 , 2016',
 'denver broncos',
 "levi ' s stadium",
 'santa clara , california',
 'super bowl l',
 '2015',
 '2016',
 'santa clara , california',
 "levi ' s stadium",
 '24 – 10',
 'february 7 , 2016',
 '2015',
 'denver broncos',
 'carolina panthers',
 'denver broncos',
 '2015',
 'denver broncos',
 "levi ' s stadium in the san francisco bay area at santa clara , california",
 'super bowl 50',
 'denver broncos',
 'cam newton',
 'eight',
 '1995',
 'arizona cardinals',
 'new england patriots',
 'arizona cardinals',
 'new england patriots',
 'new england patriots',
 'four',
 'cam newton',
 '15 – 1',
 'cam newton',
 '12 – 4 record',
 'four',
 'new england patriots',
 'cam newton',
 'arizona cardinals',
 'eight',
 'new england pat

In [441]:
true_answers=[]
for ans in datasetv:
    true_answers.append(ans["answers"]["text"][0])
    # print(ans["answers"]["text"])


In [442]:
true_answers

['Denver Broncos',
 'Carolina Panthers',
 'Santa Clara, California',
 'Denver Broncos',
 'gold',
 '"golden anniversary"',
 'February 7, 2016',
 'American Football Conference',
 '"golden anniversary"',
 'American Football Conference',
 'February 7, 2016',
 'Denver Broncos',
 "Levi's Stadium",
 'Santa Clara',
 'Super Bowl L',
 '2015',
 '2015',
 'Santa Clara',
 "Levi's Stadium",
 '24–10',
 'February 7, 2016',
 '2015',
 'Denver Broncos',
 'Carolina Panthers',
 'Denver Broncos',
 '2015',
 'Denver Broncos',
 'Santa Clara, California.',
 'Super Bowl',
 'Denver Broncos',
 'Cam Newton',
 '8',
 '1995',
 'Arizona Cardinals',
 'New England Patriots',
 'Arizona Cardinals',
 'New England Patriots',
 'New England Patriots',
 'four',
 'Cam Newton',
 '15–1',
 'Cam Newton',
 '12–4',
 '4',
 'New England Patriots',
 'Cam Newton',
 'Arizona Cardinals',
 '2',
 'New England Patriots',
 'Cam Newton',
 'New England Patriots',
 'Arizona Cardinals',
 'Cam Newton',
 'Arizona Cardinals',
 '1995.',
 'Von Miller',
 

In [443]:
predictions[5:10],true_answers[5:10]

(['golden anniversary',
  'february 7 , 2016',
  'american football conference',
  'golden anniversary',
  'american football conference'],
 ['"golden anniversary"',
  'February 7, 2016',
  'American Football Conference',
  '"golden anniversary"',
  'American Football Conference'])

In [444]:
from sklearn.metrics import f1_score

def exact_match(pred, true):
    return int(pred == true)

def evaluate(predictions, true_answers):
    em = 0  # Exact Match
    f1 = 0  # F1 Score

    for pred, true in zip(predictions, true_answers):
        em += exact_match(pred, true)
        f1 += f1_score([true], [pred], average="micro")

    total_examples = len(predictions)
    em /= total_examples
    f1 /= total_examples

    return em, f1



em_score, f1_score = evaluate(predictions, true_answers)
print(f"Exact Match: {em_score}")
print(f"F1 Score: {f1_score}")

Exact Match: 0.26
F1 Score: 0.26


In [445]:
def token_overlap_similarity(pred, true):
    pred_tokens = set(pred.lower().split())
    true_tokens = set(true.lower().split())
    
    intersection = pred_tokens.intersection(true_tokens)
    union = pred_tokens.union(true_tokens)
    
    if len(union) == 0:
        return 0.0
    
    similarity = len(intersection) / len(union)
    return similarity

def evaluate(predictions, true_answers):
    similarity_scores = []

    for pred, true in zip(predictions, true_answers):
        similarity = token_overlap_similarity(pred, true)
        similarity_scores.append(similarity)

    average_similarity = sum(similarity_scores) / len(similarity_scores)

    return average_similarity



similarity_score = evaluate(predictions, true_answers)
print(f"Similarity Score: {similarity_score}")


Similarity Score: 0.7645396294881591
